In [1]:
"""
We will cover how to build a neural network in TensorFlow 2.0
using the Keras Sequential and Functional API. We also take
a look at other details like how we obtain the output features
from specific layers and obtaining summaries of the model etc.
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
"""
Train Dataset = x_train + y_train
x_train = all the elements except ground truth column (classification column)
y_train = only the classification column (for example survival column YES or NO)

x_test = all the elements except ground truth column (classification column)
y_test = only the classification column (for example survival column YES or NO)
"""



(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [3]:
"""
We have (60000, 28, 28) 60000 images where all they are 28 by 28.
We are going to send them to a neural network, so we need to flatten
them so we have only one long column with those feature values.

In order to do the above we are going to reshape the x_train and x_test
"""

# -1 means keep whatever the value is on that dimension, in this
# case 60000. And we are going to flatten the two 28 dimemsions.
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0 # 32 bytes = 256 bits
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0 # 32 bytes = 256 bits

In [4]:
print (x_train.shape)
print (x_test.shape)

(60000, 784)
(10000, 784)


## Creating Neural Network using Sequential API

In [5]:
"""
Creating a neural network using sequential API of Keras

Sequential API (Very convenient, now very flexible)
"""

model = keras.Sequential(
    [
        keras.Input(shape=28*28),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10) # Output layer don't need activation function
        
    ]   
)

# I must have keras.Input in order to print model summary
# at this point. Otherwise I must put it after model.fit 
# at the end of this block.
print(model.summary())

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model.fit(x_train, y_train, batch_size=32, epochs=5)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1875/1875 [==============================] - 18s 9ms/step - loss: 0.1861 - accuracy: 0.9425
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0811 - accuracy: 0.9747
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0534 - accuracy: 0.9833
Epo

[0.08310019969940186, 0.9765999913215637]

In [6]:
"""
Adding layers and print model summary in the middle.
So we can check the network at any stage.
"""

model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
print(model.summary())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
Total params: 401,920
Trainable params: 401,920
Non-trainable params: 0
_________________________________________________________________
None


## Creating Neural Network using Functional API

In [7]:
"""
Creating a neural network using functional API of Keras

Functional API (A bit more flexible)
"""

inputs = keras.Input(shape=(28*28), name='Input')
x = layers.Dense(512, activation='relu', name='1st_layer')(inputs)
x = layers.Dense(256, activation='relu',  name='2nd_layer')(x)
outputs = layers.Dense(10, activation='softmax', name='Output')(x) #softmax normalizes the output.
# takes the inputs & outputs we are define and builds the model.
model = keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model.fit(x_train, y_train, batch_size=32, epochs=5)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 784)]             0         
                                                                 
 1st_layer (Dense)           (None, 512)               401920    
                                                                 
 2nd_layer (Dense)           (None, 256)               131328    
                                                                 
 Output (Dense)              (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.1886 - accuracy: 0.9425
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.07

[0.07410307973623276, 0.9790999889373779]